In [1]:
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer,
    pipeline,
    logging
)

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    '../assistant_sft/outputs/phi_1_5_oasst_guanaco_qlora/best_model/',
).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(
    '../assistant_sft/outputs/phi_1_5_oasst_guanaco_qlora/best_model/'
)

/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
print(tokenizer.eos_token)

<|endoftext|>


In [4]:
logging.set_verbosity(logging.CRITICAL)

In [5]:
template = "### Human: {prompt}### Assistant:"
eos_string = tokenizer.eos_token
history = None

In [6]:
while True:
    question=input("Question: ")

    prompt = history + '\n' + template.format(prompt=question) if history is not None else template.format(prompt=question)

    prompt_tokenized=tokenizer(prompt, return_tensors='pt').to('cuda')['input_ids']
    output_tokenized = model.generate(
        input_ids=prompt_tokenized, 
        max_length=len(prompt_tokenized[0])+400,
        temperature=0.7,
        top_k=40,
        top_p=0.1,
        eos_token_id=tokenizer.eos_token_id,
    )
    answer = tokenizer.decode(output_tokenized[0][len(prompt_tokenized[0]):]).strip()
    
    if eos_string in answer:
        answer = answer.split(eos_string)[0].strip()
    if '###' in answer:
        answer = answer.split('###')[0].strip()

    history = '\n'.join([prompt, answer, eos_string])
    print(f"ANSWER: {answer}\n")
    # print(f"HISTORY: {history}\n")
    print('#' * 50)

Question:  hi


/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `40` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


ANSWER: Hello! How can I help you today?

##################################################


Question:  tell me your family history


ANSWER: I am sorry, but I cannot provide you with information about my family history. However, I can provide you with information about the history of the Open Assistant chatbot.

The Open Assistant chatbot was first introduced in 2017 by Open Assistant, a project by the Open Assistant team. The chatbot was designed to be a conversational assistant that could interact with users in a natural and conversational way. The chatbot was released publicly in December 2017 and has since become one of the most popular chatbots on the internet.

The Open Assistant chatbot has been used for a variety of tasks, including customer service, technical support, and educational purposes. The chatbot has also been used for research and development, and has been used to train other chatbots.

The Open Assistant chatbot has been praised for its natural language understanding and its ability to interact with users in a conversational way. The chatbot has also been praised for its ability to learn and adap

KeyboardInterrupt: Interrupted by user